<a href="https://colab.research.google.com/github/jamglez/ML-FreeCodeCamp/blob/main/Copia_de_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
!pip install seaborn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

# **Clean Data**


In [ ]:
dataset.isna().sum()

In [ ]:
dataset.head()

In [ ]:
dataset.dtypes

In [ ]:
#Convert categorical data to numbers
dataset['sex'] =  np.where(dataset['sex']=='female', 1, 0)
dataset['smoker'] =  np.where(dataset['smoker']=='yes', 1, 0)
dataset['region'] = dataset['region'].map({'southwest': 'southwest', 'southeast': 'southeast', 'northwest': 'northwest', 'northeast': 'northeast'}) #One hot encoding
dataset = pd.get_dummies(dataset, columns=['region'], prefix='', prefix_sep='')


In [ ]:
dataset.head()

# **Inspect the data**

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
sns.pairplot(train_dataset[['age','bmi','children']],diag_kind="kde")

In [ ]:
train_stats = train_dataset.describe().transpose()
train_stats

# **Normalize data**

In [ ]:
# Creating a normalizer layer
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))
print(normalizer.mean.numpy())

# **Build the model**

In [ ]:
def build_model():
  model = keras.Sequential([
      normalizer,
      layers.Dense(16, activation="relu"), #16
      layers.Dense(32, activation="relu"), #32
      layers.Dense(1)
  ])

  model.compile(
      loss='mse',
      optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.025),
      metrics=['mae','mse']
  )

  return model


model = build_model()

In [ ]:
model.summary()

In [ ]:
#Check if the shape is correct
example_batch = train_dataset[:10]
example_result = model.predict(example_batch)
example_result

# **Train the model**

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if epoch % 100 == 0: print('')
    print('.',end='')

EPOCHS = 500

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)

history = model.fit( train_dataset, train_labels,  epochs=EPOCHS, validation_split = 0.2, verbose=2,
    callbacks=[early_stop,PrintDot()]
)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
